<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Finance with Python

**Chapter 06 &mdash; Dynamic Economy**

## Binomial Model

In [ ]:
!git clone https://github.com/tpq-classes/finance_with_python.git
import sys
sys.path.append('finance_with_python')


In [ ]:
import math
import numpy as np

In [ ]:
S0 = 36.
K = 40.
r = 0.06
T = 1.0
sigma = 0.2

In [ ]:
m = 4
dt = T / m
df = math.exp(-r * dt)
up = math.exp(sigma * math.sqrt(dt))
down = 1 / up

In [ ]:
q = (1 / df - down) / (up - down)

### Binomial Option Pricing based on Python Loops

In [ ]:
S = np.zeros((m + 1, m + 1))
S

In [ ]:
S[0, 0] = S0
S

In [ ]:
z = 1
for t in range(1, m + 1):
    for i in range(0, z):
        S[i, t] = S[i, t - 1] * up
        S[i + 1 ,t] = S[i, t - 1] * down
    z += 1

In [ ]:
np.set_printoptions(formatter=
        {'float_kind': lambda x: '%7.3f' % x})

In [ ]:
S

In [ ]:
h = np.zeros_like(S)

In [ ]:
z = 1
for t in range(0, m + 1):
    for i in range(0, z):
        h[i, t] = max(K - S[i, t], 0)
    z += 1

In [ ]:
h

In [ ]:
V = np.zeros_like(S)
V[:, -1] = h[:, -1]
V

In [ ]:
m

In [ ]:
# European option pricing
z = 0
for t in range(m - 1, -1, -1):
    for i in range(0, m - z):
        V[i, t] = df * (q * V[i, t + 1] +
                    (1-q) * V[i + 1, t + 1])
    z += 1

In [ ]:
V

In [ ]:
V[0, 0]

In [ ]:
# American option pricing
z = 0
for t in range(m - 1, -1, -1):
    for i in range(0, m-z):
        V[i, t] = df * (q * V[i, t + 1] +
                  (1 - q) * V[i + 1, t + 1])
        # checking whether early exercise is better than not exercising
        V[i, t] = max(h[i, t], V[i, t]) 
    z += 1

In [ ]:
V

In [ ]:
V[0, 0]

### Vectorized Implementation with NumPy

In [ ]:
u = np.arange(m + 1)
u

In [ ]:
u ** 2

In [ ]:
2 ** u

In [ ]:
u = np.resize(u, (m + 1, m + 1))
u

In [ ]:
d = u.T
d

In [ ]:
(u - 2 * d)

In [ ]:
S = S0 * np.exp(sigma * math.sqrt(dt) * (u - 2 * d))
S

In [ ]:
h = np.maximum(K - S, 0)
h

In [ ]:
V = h.copy()

In [ ]:
# European option pricing
for t in range(m - 1, -1, -1):
    V[0:-1, t] = df * (q * V[:-1, t + 1] +
                   (1-q) * V[1:, t + 1])

In [ ]:
V[0, 0]

In [ ]:
# American option pricing
for t in range(m - 1, -1, -1):
    V[0:-1, t] = df * (q * V[:-1, t + 1] +
                   (1-q) * V[1:, t + 1])
    V[:, t] = np.maximum(h[:, t], V[:, t])

In [ ]:
V

In [ ]:
V[0, 0]

### Speed Comparison

In [ ]:
m = 500
dt = T / m
df = math.exp(-r * dt)
up = math.exp(sigma * math.sqrt(dt))
down = 1 / up
q = (1 / df - down) / (up - down)
q

In [ ]:
def binomial_looping():
    # stock price simulation in binomial tree
    S = np.zeros((m + 1, m + 1))
    S[0, 0] = S0
    z = 1
    for t in range(1, m + 1):
        for i in range(0, z):
            S[i, t] = S[i, t - 1] * up
            S[i + 1 ,t] = S[i, t - 1] * down
        z += 1
    # inner value calculation
    h = np.zeros_like(S)
    z = 1
    for t in range(0, m + 1):
        for i in range(0, z):
            h[i, t] = max(K - S[i, t], 0)
        z += 1
    # American option pricing
    V = np.zeros_like(S)
    V[:, -1] = h[:, -1]
    z = 0
    for t in range(m - 1, -1, -1):
        for i in range(0, m - z):
            V[i, t] = df * (q * V[i, t + 1] +
                      (1 - q) * V[i + 1, t + 1])
            V[i, t] = max(h[i, t], V[i, t])
        z += 1
    return V[0, 0]

In [ ]:
%time binomial_looping()

In [ ]:
%timeit binomial_looping()

In [ ]:
def binomial_vectorization():
    u = np.arange(m + 1)
    u = np.resize(u, (m + 1, m + 1))
    d = u.T
    # stock price simulation
    S = S0 * np.exp(sigma * math.sqrt(dt) * (u - 2 * d))
    # inner value calculation
    h = np.maximum(K - S, 0)
    # American option pricing
    V = h.copy()
    for t in range(m-1, -1, -1):
        V[0:-1, t] = df * (q * V[:-1, t + 1] +
                       (1-q) * V[1:, t + 1])
        V[:, t] = np.maximum(h[:, t], V[:, t])
    return V[0, 0]

In [ ]:
%time binomial_vectorization()

In [ ]:
%timeit binomial_vectorization()

In [ ]:
469 / 11.7  # speed-up factor of vectorized code

## Monte Carlo Simulation

In the Black-Scholes-Merton (1973) model based on the geometric Brownian motion, the future price $S_t$ of the risky stock is given in a discrete simulation context by

$$S_t = S_{t - \Delta t} \cdot \exp \left(\left(r - \frac{\sigma^2}{2} \right)\Delta t + \sigma \sqrt{\Delta t} z \right)$$

$S_0$ is the initial stock price, $r$ the risk-free short rate, $\sigma$ the volatility factor, $T>0$ a future point in time and $z$ a standard normally distributed rv. $\Delta t$ is the homogeneous time interval.

In [ ]:
S0 = 36.
K = 40.
r = 0.06
T = 1.0
sigma = 0.2

### Simulating the Stock Price Process

In [ ]:
M = 100
I = 50000

In [ ]:
dt = T / M
dt

In [ ]:
df = math.exp(-r * dt)
df

In [ ]:
np.random.seed(100)

In [ ]:
rn = np.random.standard_normal((M + 1, I))
rn.round(2)

In [ ]:
S = np.zeros_like(rn)
S[0] = S0
S

In [ ]:
for t in range(1, M + 1):
    S[t] = S[t - 1] * np.exp((r - sigma ** 2 / 2) * dt +
                           sigma * math.sqrt(dt) * rn[t])

In [ ]:
S

In [ ]:
from pylab import mpl, plt
plt.style.use('seaborn-v0_8')
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['savefig.dpi'] = 300

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(S[:, :10]);

In [ ]:
ST = S[-1]
plt.figure(figsize=(10, 6))
plt.hist(ST, bins=50, color='b', label='frequency');
plt.axvline(ST.mean(), color='r', label='mean')
plt.axvline(ST.mean() + ST.std(), color='y', label='sd up')
plt.axvline(ST.mean() - ST.std(), color='y', label='sd down')
plt.legend(loc=0);

In [ ]:
S0 * math.exp(r * T)

In [ ]:
S[-1].mean()

### European Option Pricing

In [ ]:
h = np.maximum(K - ST, 0)
h

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(h, color='b', bins=50);

In [ ]:
math.exp(-r * T) * h.mean()  # Monte Carlo estimator of European option value

### American Option Pricing

In [ ]:
h = np.maximum(K - S, 0)
h

In [ ]:
# Least-Squares Monte Carlo Valuation (LSM algorithm)
V = h[-1]
for t in range(M - 1, 0, -1):
    reg = np.polyfit(S[t], df * V, deg=5)
    C = np.polyval(reg, S[t])
    V = np.where(h[t] > C, h[t], df * V)

In [ ]:
df * V.mean()

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>